In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 6.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=90bf8c6529a4cb9332d8d176f82447c363af172e8f9e605dc42b66bfdeec1eff
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=03618c8380a1c3e5aefe30ad195d0aa8a090c0d12d36ca263482744df064e984
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
import nltk
nltk.download('stopwords')
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# LoadDataset
df=pd.read_csv("/content/UScomments.csv",error_bad_lines=False)
print(df.video_id.unique())
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


['XpVt6Z1Gjjo' 'cLdxuaxaQwc' 'WYYvHb03Eog' 'sjlHnJvXdQs' 'cMKX2tE5Luk'
 '8wNr-NQImFg' '_HTXMhKWqnA' '_ANP3HR1jsM' 'zgLtEob6X-Q' 'Ayb_2qbZHm4'
 'CsdzflTXBVQ' 'l864IBj7cgw' '4MkC65emkG4' 'vu_9muoxT50' '1L7JFN7tQLs'
 'ZQK1F0wz6z4' 'T_PuZBdT2iM' 'w8fAellnPns' 'UCrBICYM0yM' '-Ifnaxi2LQg'
 'B7YaMkCl3XA' '5ywKal6-anc' '4Yue-q9Jdbk' 'JhA1Wi9mrns' 'EVp4-qjWVJE'
 'LcZ2AuvxXNA' 'MdzGZv3zQ-U' '1ZNZY-gd3K0' '2wxyDrfwlXQ' 'QBGaO89cBMI'
 'ql0Op1VcELw' 'JO7X9ZPoAp8' 'GGm0FQ6i74U' 'a7Sf_H2cFdM' 'oDIDZ9EmQfA'
 'zAtHxJvSczA' 'l_uNfGY3v8E' 'Rew50oYjRqA' 'WWexI9YiLSc' 'LDcm6twPEJA'
 'O78Lpo4ctSE' 'qJJHhVf3_ZM' '3yoGE3v4A8w' 'zTjcPeb2Gwg' 'hQbeB0YXLhs'
 'L3f7_y9UPh4' '6vGg-jJl30A' '9CILSvf1snQ' 'WoPtuVbaSKQ' '62cWuFjzOgs'
 'NzRuDD0iYC0' 'eM_FR7I2Ttw' 'eoArC99zm1M' 'IMG0u0_cKNc' 'bAon04ZJhHE'
 'LTnl36-0-zU' 'EmjMWi0MBXQ' 'c7ymY8QEB2g' 'Ez95rPffd8k' '08RHSZvixec'
 'zAXn-v9RdXo' '_SRNenu07Lw' 'iXexmZm3_j8' 'GpAuCG6iUcA' 'eRDxDEhEgNY'
 'AZTGeQ5msaU' 'yiblDn4JGfQ' 'xKXxJcf6_IE' 'YQYslaA1Lus' 'k9bYO7uSnrg'
 '1nEp

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0.0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0.0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0.0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0.0
4,XpVt6Z1Gjjo,trending 😉,3,0.0


In [ ]:
#Convert to list 
data = df.comment_text.values.tolist()  

In [ ]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['logan', 'paul', 'it', 'yo', 'big', 'day']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['logan_paul', 'it', 'yo', 'big', 'day']


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['big', 'day']]


In [ ]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [ ]:
da_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [ ]:
# Print the keyword of topics
pprint(da_model.print_topics())
doc_lda = da_model[corpus]

[(0,
  '0.095*"look" + 0.033*"way" + 0.025*"foundation" + 0.021*"light" + '
  '0.020*"wear" + 0.017*"amazing" + 0.016*"long" + 0.015*"review" + '
  '0.015*"makeup" + 0.014*"stuff"'),
 (1,
  '0.050*"hurricane" + 0.023*"body" + 0.018*"weather" + 0.017*"woman" + '
  '0.015*"sometimes" + 0.015*"bring" + 0.014*"hear" + 0.014*"bit" + '
  '0.013*"weird" + 0.013*"damn"'),
 (2,
  '0.126*"get" + 0.040*"also" + 0.037*"need" + 0.026*"man" + 0.013*"happy" + '
  '0.012*"memory" + 0.011*"money" + 0.010*"stay" + 0.010*"rain" + '
  '0.009*"strong"'),
 (3,
  '0.082*"go" + 0.074*"feel" + 0.043*"time" + 0.033*"come" + 0.027*"happen" + '
  '0.022*"life" + 0.021*"bad" + 0.021*"always" + 0.021*"storm" + '
  '0.020*"world"'),
 (4,
  '0.070*"good" + 0.029*"friend" + 0.025*"eye" + 0.024*"job" + 0.021*"head" + '
  '0.021*"leave" + 0.018*"use" + 0.018*"keep" + 0.018*"category" + '
  '0.017*"sure"'),
 (5,
  '0.083*"really" + 0.078*"make" + 0.057*"thank" + 0.037*"never" + '
  '0.025*"year" + 0.019*"girl" + 0.019*"a

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', da_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=da_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt


Perplexity:  -9.241598598585096

Coherence Score:  0.3705816459160394


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(da_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.354774  0.063618       1        1  12.670068
8     -0.125967  0.304005       2        1  12.203999
2     -0.054559 -0.212238       3        1  12.020662
4     -0.035649 -0.022981       4        1  10.614339
7     -0.031008 -0.048274       5        1  10.489121
9     -0.025731 -0.019504       6        1  10.467743
0     -0.022674 -0.020565       7        1   8.943341
5     -0.018842 -0.019138       8        1   8.007859
3     -0.019573 -0.014479       9        1   7.598177
6     -0.020771 -0.010445      10        1   6.984691, topic_info=          Term        Freq       Total Category  logprob  loglift
93          go  702.000000  702.000000  Default  30.0000  30.0000
162      video  703.000000  703.000000  Default  29.0000  29.0000
69      people  592.000000  592.000000  Default  28.0000  28.0000
26         get  613.000000  613.000000  Default  27.0000  27.0000
395       feel  606.000000  606.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
561   struggle   29.800348   30.513104  Topic10  -4.9897   2.6378
562      voice   27.083791   27.793323  Topic10  -5.0853   2.6356
671      phone   25.334348   26.043749  Topic10  -5.1521   2.6338
424       term   22.911895   23.621816  Topic10  -5.2526   2.6309
1654       son   23.463636   26.477747  Topic10  -5.2288   2.5406

[333 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1735      4  0.993410       act
185       1  0.997671  actually
3452      8  0.984528     alive
70        5  0.986779    almost
804       3  0.985279   already
...     ...       ...       ...
381       2  0.996571     wrong
16        1  0.998574      year
638       2  0.977707     young
22        3  0.993081   youtube
1117      8  0.979624  youtuber

[303 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 9, 3, 5, 8, 10, 1, 6, 4, 7])